In [1]:
import pandas as pd
from __future__ import print_function
import importlib.util
import math
import os
import sys
import numpy as np

from collections import defaultdict
from rdkit import Chem

# If you don't want to draw your molecule these aren't necessary
from rdkit.Chem import Draw
from rdkit.Chem.Draw import DrawingOptions
from rdkit.Chem.Draw import IPythonConsole

from rdkit.Chem import AllChem
from rdkit.Chem import RDConfig
sys.path.append(os.path.join(RDConfig.RDContribDir,'ChiralPairs'))
from ChiralDescriptors import determineAtomSubstituents

In [2]:
from rdkit import Chem
suppl = Chem.SDMolSupplier("data\\qm9\\raw\\gdb9.sdf", removeHs=False,
                                   sanitize=True)

In [3]:
# TODO: Implement logic to determine e.g. whether repeats of CCCCC are cyclopentyl and pentyl or two of either
def GetChemicalNonequivsNew(atom, themol):
    num_unique_substituents = 0
    # 8 should do; I don't know of an atom with more bonds than Rhenium in [Re_{2}Cl_{8}]^{2-}
    substituents = [[],[],[],[],[],[],[],[]]
    substituents_just_atom_labels = [[],[],[],[],[],[],[],[]]
    for item,key in enumerate(ChiralDescriptors.determineAtomSubstituents(atom.GetIdx(), themol, Chem.GetDistanceMatrix(themol))[0]):
        for subatom in ChiralDescriptors.determineAtomSubstituents(atom.GetIdx(), themol, Chem.GetDistanceMatrix(themol))[0][key]:
            substituents[item].append(themol.GetAtomWithIdx(subatom))
            substituents_just_atom_labels[item].append(themol.GetAtomWithIdx(subatom).GetSymbol())
    # At this point in the program we have the four-membered list of substituents.
    # If the chains are of different lengths then it is impossible for them to be chemically equivalent.
    # Therefore, our first hack will be to check the number of unique substituents and their lengths against each other.
    # But let's rearrange the list so that we have only the substituents that actually exist -- the rest are hydrogen.
    num_unique_substituents = len(set(tuple(tuple(substituent) for substituent in substituents_just_atom_labels if substituent)))
    return num_unique_substituents

def GetBottcherLocalDiversity(atom):
    neighbors = []
    for neighbor in atom.GetNeighbors():
        neighbors.append(str(neighbor.GetSymbol()))
    if atom.GetSymbol() in set(neighbors):
        return len(set(neighbors))
    else:
        return len(set(neighbors))+1

def GetNumIsomericPossibilities(atom):
    try:
        if(atom.GetProp('_CIPCode')):
            return 2
    except:
        return 1
def GetNumValenceElectrons(atom):
    valence = {1: ['H', 'Li', 'Na', 'K', 'Rb', 'Cs', 'Fr'], # Alkali Metals
               2: ['Be', 'Mg', 'Ca', 'Sr', 'Ba', 'Ra'], # Alkali Earth Metals
               #transition metals???
               3: ['B', 'Al', 'Ga', 'In', 'Tl', 'Nh'], #
               4: ['C', 'Si', 'Ge', 'Sn', 'Pb', 'Fl'],
               5: ['N', 'P', 'As', 'Sb', 'Bi', 'Mc'], # Pnictogens
               6: ['O', 'S', 'Se', 'Te', 'Po', 'Lv'], # Chalcogens
               7: ['F', 'Cl', 'Br', 'I', 'At', 'Ts'], # Halogens
               8: ['He', 'Ne', 'Ar', 'Kr', 'Xe', 'Rn', 'Og']} # Noble Gases
    for k in valence:
        if atom.GetSymbol() in valence[k]:
            return k
    return 0


def GetBottcherBondIndex(atom):
    b_sub_i_ranking = 0
    bonds = []
    for bond in atom.GetBonds():
        bonds.append(str(bond.GetBondType()))
    for bond in bonds:
        if bond == 'SINGLE':
            b_sub_i_ranking += 1
        if bond == 'DOUBLE':
            b_sub_i_ranking += 2
        if bond == 'TRIPLE':
            b_sub_i_ranking += 3
def GetBottcherBondIndex(atom):
    b_sub_i_ranking = 0
    bonds = []
    for bond in atom.GetBonds():
        bonds.append(str(bond.GetBondType()))
    for bond in bonds:
        if bond == 'SINGLE':
            b_sub_i_ranking += 1
        if bond == 'DOUBLE':
            b_sub_i_ranking += 2
        if bond == 'TRIPLE':
            b_sub_i_ranking += 3
    if 'AROMATIC' in bonds:
    # This list can be expanded as errors arise.
        if atom.GetSymbol() == 'C':
            b_sub_i_ranking += 3
        elif atom.GetSymbol() == 'N' or atom.GetSymbol() == 'S':
            b_sub_i_ranking += 2
    return b_sub_i_ranking

def remove_values_from_list(the_list, val):
   return [value for value in the_list if value != val]

def GetTheAtoms(themol, symmetries, halflings=False):
    # We need to see what the count is for each CIPCode and adjust
    # Say our count of _CIPCode 0 is six, then we need to append that atom
    # to compute_full_complexity three times.
    # If it is five, we need to append to compute_full_complexity twice and
    # compute_half_complexity once.
    # In both cases, we need to remove their CIPCode frem the list of CIPcodes
    # so that no other atom with that CIPCode gets counted.
    # We leverage the fact that each atom with the same CIPCode is by def. the
    # same to accomplish this.
    compute_full_complexity = []
    compute_half_complexity = []
    for atom in themol.GetAtoms():
        number_of_occurrences = symmetries.count(atom.GetProp('_CIPRank'))
        if number_of_occurrences == 1 :
            compute_full_complexity.append(atom)
            continue;
        else:
            number_of_times_full = number_of_occurrences // 2;
            number_of_times_half = number_of_occurrences % 2;
            for i in range(0,number_of_times_full):
                compute_full_complexity.append(atom)
            for i in range(0,number_of_times_half):
                compute_half_complexity.append(atom)
            symmetries = remove_values_from_list(symmetries, atom.GetProp('_CIPRank'))
            continue;

    if halflings:
        return compute_half_complexity
    else:
        return compute_full_complexity



In [4]:
def GetBottcherComplexity(themol,debug=False):
    complexity = 0
    ## Assign CIPCodes to each atom in the molecule
    Chem.AssignStereochemistry(themol,cleanIt=True,force=True,flagPossibleStereoCenters=True)
    ## Append each atom's CIPCode to a list
    CIPCodes = []
    for atom in themol.GetAtoms():
        CIPCodes.append(atom.GetProp('_CIPRank'))
    atoms = GetTheAtoms(themol, CIPCodes)
    half_atoms = GetTheAtoms(themol, CIPCodes, True)
    if debug:
        print("FULL ATOMS")
    for atom in atoms:
        d = GetChemicalNonequivsNew(atom, themol)
        e = GetBottcherLocalDiversity(atom)
        s = GetNumIsomericPossibilities(atom)
        V = GetNumValenceElectrons(atom)
        b = GetBottcherBondIndex(atom)
        complexity += d*e*s*math.log(V*b,2)
        if debug:
            PrintDebugInfo(str(atom.GetSymbol()), str(atom.GetProp('_CIPRank')), d, e, s, V, b)
    if half_atoms:
        if debug:
            print("HALF ATOMS")
        for atom in half_atoms:
            d = GetChemicalNonequivsNew(atom, themol)
            e = GetBottcherLocalDiversity(atom)
            s = GetNumIsomericPossibilities(atom)
            V = GetNumValenceElectrons(atom)
            b = GetBottcherBondIndex(atom)
            complexity += (d*e*s*math.log(V*b,2)/2)
            if debug:
                PrintDebugInfo(str(atom.GetSymbol()), str(atom.GetProp('_CIPRank')), d, e, s, V, b)
    if debug:
        print('Current Complexity Score: ' + str(complexity))
        return
    return complexity

In [5]:
from ChiralDescriptors import determineAtomSubstituents
import ChiralDescriptors 

In [9]:
def findmol(name,suppl):
    for mol in suppl:
        molname=mol.GetProp('_Name')
        if name==molname:
            return mol
    return None

In [6]:
GetBottcherComplexity(suppl[0])

8.0

In [8]:
df1=pd.read_csv("homo.csv")
#df2=pd.read_csv("lumo.csv")

In [11]:
names=df1["name"]
complexities=[]
for name in names:
    mol=findmol(name,suppl)
    complexity=GetBottcherComplexity(suppl[0])
    complexities.append(complexity)


AttributeError: 'NoneType' object has no attribute 'GetProp'

In [12]:
complexities=[i for i in range(len(names))]

In [14]:
df1["compl"]=complexities
df2["compl"]=complexities

In [ ]:
df1.to_csv("homo.csv")
df2.to_csv("lumo.csv")